In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
import MDAnalysis as mda
%matplotlib inline

In [ ]:
work_dir = '/home/eva/Documents/explicit-solvent-sims/ff19SB-OPC/hex_L24E/pH6/'
top_dir = '/home/eva/Documents/structures/explicit-fromh++/ff19SB-OPC/hex_L24E/pH6/'
system_code = 'CC-Hex*-L24E-hex, pH 6'

Loading the imaged trajectory, after it has been post-processed by cpptraj

In [ ]:
traj_imaged = md.load(work_dir+'imaged_output.dcd', top = top_dir+'6q5j_pH6_explicit.prmtop')

In [ ]:
#traj_imaged[::10].save_dcd(work_dir+'imaged_output_every10thframe.dcd')

In [ ]:
#traj_skip10 = md.load(work_dir+'imaged_output_every10thframe.dcd', top = top_dir+'6q5j_pH8.5_explicit.prmtop')

In [ ]:
traj_imaged.n_frames

Deleting water molecules from the imaged trajectory

In [ ]:
protein_traj = traj_imaged.remove_solvent(inplace=False)

In [ ]:
protein_traj

In [ ]:
protein_traj.topology

In [ ]:
protein_traj.save_dcd(work_dir+'protein_traj.dcd')

In [ ]:
protein_traj_trunc = md.iterload(work_dir+'protein_traj.dcd', top = top_dir+'0.15_80_10_pH6_6q5j_mod_noh_rename_noconect.genpdb.cpptraj.pdb', chunk = 0, skip = 1000)

for tr in protein_traj_trunc:
    protein_traj_short = tr
print(protein_traj_short)

In [ ]:
#imaged_traj = traj.image_molecules(inplace=False)
#imaged_traj.save_dcd(work_dir+'imaged_output.dcd')

In [ ]:
t = np.arange(0.0, 200.0, 0.02)
t_trunc = np.arange(20.0, 200.0, 0.02)

Loading replicate runs and saving the trajectories without solvent

In [ ]:
work_dir_2 = work_dir+'replicate-2/'
top_dir_2 = top_dir

traj_2 = md.load(work_dir_2+'imaged_output.dcd', top = top_dir_2+'6q5j_pH6_explicit.prmtop')

protein_traj_2 = traj_2.remove_solvent(inplace=False)
protein_traj_2.save_dcd(work_dir_2+'protein_traj.dcd')

#skip number needs to be changed according to timestep
protein_traj_trunc_2 = md.iterload(work_dir_2+'protein_traj.dcd', top = top_dir_2+'0.15_80_10_pH6_6q5j_mod_noh_rename_noconect.genpdb.cpptraj.pdb', chunk = 0, skip = 1000)

for tr_2 in protein_traj_trunc_2:
    protein_traj_short_2 = tr_2
print(protein_traj_short_2)

In [ ]:
work_dir_3 = work_dir+'replicate-3/'
top_dir_3 = top_dir

traj_3 = md.load(work_dir_3+'imaged_output.dcd', top = top_dir_3+'6q5j_pH6_explicit.prmtop')

protein_traj_3 = traj_3.remove_solvent(inplace=False)
protein_traj_3.save_dcd(work_dir_3+'protein_traj.dcd')

#skip number needs to be changed according to timestep
protein_traj_trunc_3 = md.iterload(work_dir_3+'protein_traj.dcd', top = top_dir_3+'0.15_80_10_pH6_6q5j_mod_noh_rename_noconect.genpdb.cpptraj.pdb', chunk = 0, skip = 1000)

for tr_3 in protein_traj_trunc_3:
    protein_traj_short_3 = tr_3
print(protein_traj_short_3)

### Structure

Separating the amino acids and the ACE-NH2 caps, and creating lists for their indexes:

In [ ]:
res_obj = protein_traj.topology.residues
res_list=[]
caps_positions =[]
all_residues=[]


for res in res_obj:
        all_residues.append(str(res))
        if res.name == 'ACE' or res.name == 'NHE':
            print('excluding the ACE and NH2 caps')
            caps_positions.append(str(res))
        else:
            res_list.append(str(res))
            
res_idx=[]
for res in res_list:
    res_idx.append(int(res[3:6]))
    
caps_idx=[]
for cap in caps_positions:
    caps_idx.append(int(cap[3:6]))
    
all_residues_idx=[]
for re in all_residues:
    all_residues_idx.append(int(re[3:6]))

In [ ]:
chain_number = int(len(caps_idx)/2)
print('the number of chains in the structure is', chain_number)

In [ ]:
ace=caps_idx[0::2]
nh2=caps_idx[1::2]
print('the ACE positions are', ace)
print('the NH2 positions are', nh2)

In [ ]:
chain_list=[]
chain=[]
chain_idx = []
all_idx=[]

for i in range(len(ace)):
    chain = all_residues[(ace[i]+1):(nh2[i])]
    chain_list.append(chain)
    chain=[]

for ch in chain_list:
    for re in ch:
        all_idx.append(int(re[3:6]))    

for i in range(len(ace)):
    chain = all_residues_idx[(ace[i]+1):(nh2[i])]
    chain_idx.append(chain)
    chain=[]

In [ ]:
chain_length = len(chain_idx[0])
print('the length of each chain is', chain_length, 'residues')

### RMSD Calculation

In [ ]:
rmsds = md.rmsd(protein_traj, protein_traj, 0, atom_indices = [a.index for a in protein_traj.topology.atoms if a.is_backbone])

In [ ]:
rmsds_100 = []
for rmsd in rmsds:
    rmsds_100.append(rmsd/(1+np.log(np.sqrt(chain_number*chain_length/100))))

RMSD plot

In [ ]:
fig,ax = plt.subplots()
ax.plot(t, rmsds)
ax.set(xlabel='t (ns)', ylabel='RMSD (nm)', title='RMSD')
#ax.grid()
#ax.set_ylim([0.0, 1.0])

fig.savefig(work_dir+'analysis/rmsd.png')
plt.show()

RMSD100 plot

In [ ]:
fig,ax = plt.subplots()
ax.plot(t, rmsds_100)
ax.set(xlabel='t (ns)', ylabel='$RMSD_{100}$ (nm)', title='$RMSD_{100}$')
#ax.grid()
ax.set_xlim([0.0, 210.0])

fig.savefig(work_dir+'analysis/rmsd_100.png')
plt.show()

RMSDs for the truncated trajectory

In [ ]:
rmsds_trunc = md.rmsd(protein_traj_short, protein_traj, 0, atom_indices = [a.index for a in protein_traj.topology.atoms if a.is_backbone])

In [ ]:
fig,ax = plt.subplots()
ax.plot(t_trunc, rmsds_trunc)
ax.set(xlabel='t (ns)', ylabel='RMSD (nm)', title='RMSD')
#ax.grid()
#ax.set_ylim([0.0, 1.0])

fig.savefig(work_dir+'analysis/rmsd-trunc.png')
plt.show()

In [ ]:
rmsds_trunc_100 = []

for rmsd in rmsds_trunc:
    rmsds_trunc_100.append(rmsd/(1+np.log(np.sqrt(chain_number*chain_length/100))))

In [ ]:
fig,ax = plt.subplots()
ax.plot(t_trunc, rmsds_trunc_100)
ax.set(xlabel='t (ns)', ylabel='$RMSD_{100}$ (nm)', title='$RMSD_{100}$')
#ax.grid()
ax.set_xlim([0.0, 200.0])
ax.set_ylim([0.0, 0.5])

fig.savefig(work_dir+'analysis/rmsd-trunc-100.png')
plt.show()

Comparisons between replicates

In [ ]:
rmsds_trunc_2 = md.rmsd(protein_traj_short_2, protein_traj_2, 0, atom_indices = [a.index for a in protein_traj_2.topology.atoms if a.is_backbone])

rmsds_trunc_2_100 =[]
for rmsd in rmsds_trunc_2:
    rmsds_trunc_2_100.append(rmsd/(1+np.log(np.sqrt(chain_number*chain_length/100))))

In [ ]:
rmsds_trunc_3 = md.rmsd(protein_traj_short_3, protein_traj_3, 0, atom_indices = [a.index for a in protein_traj_3.topology.atoms if a.is_backbone])

rmsds_trunc_3_100 = []
for rmsd in rmsds_trunc_3:
    rmsds_trunc_3_100.append(rmsd/(1+np.log(np.sqrt(chain_number*chain_length/100))))

In [ ]:
fig,ax = plt.subplots()
ax.plot(t_trunc, rmsds_trunc, label = 'replicate 1')
ax.plot(t_trunc, rmsds_trunc_2, label = 'replicate 2', color = 'darkorange')
ax.plot(t_trunc, rmsds_trunc_3, label = 'replicate 3', color = 'yellowgreen')

ax.set(xlabel='t (ns)', ylabel='RMSD (nm)', title='')
#ax.grid()
ax.legend(bbox_to_anchor=(1,1), loc='upper left')
#ax.set_ylim([0.0, 1.0])

fig.savefig(work_dir+'analysis/rmsd-trunc-comparison-between-replicates.png', bbox_inches="tight")

plt.show()

In [ ]:
fig,ax = plt.subplots()
ax.plot(t_trunc, rmsds_trunc_100, label = 'replicate 1')
ax.plot(t_trunc, rmsds_trunc_2_100, label = 'replicate 2', color = 'darkorange')
ax.plot(t_trunc, rmsds_trunc_3_100, label = 'replicate 3', color = 'yellowgreen')

ax.set(xlabel='t (ns)', ylabel='$RMSD_{100}$ (nm)', title=system_code)
#ax.grid()
ax.legend(bbox_to_anchor=(1,1), loc='upper left')
ax.set_xlim([0.0, 200.0])
#ax.set_ylim([0.0, 1.0])

fig.savefig(work_dir+'analysis/rmsd-trunc-100-comparison-between-replicates.png', bbox_inches="tight")

plt.show()

Conversion of all RMSD to Angstroms

In [ ]:
rmsds_trunc_A_100=[]
rmsds_trunc_2_A_100=[]
rmsds_trunc_3_A_100=[]

for i in rmsds_trunc_100:
    rmsds_trunc_A_100.append(10*i)

for i in rmsds_trunc_2_100:
    rmsds_trunc_2_A_100.append(10*i)

for i in rmsds_trunc_3_100:
    rmsds_trunc_3_A_100.append(10*i)

In [ ]:
rmsd_100_avg = []

for i in range(len(rmsds_trunc_A_100)):
    rmsd_100_avg.append((rmsds_trunc_A_100[i]+rmsds_trunc_2_A_100[i]+rmsds_trunc_3_A_100[i])/3)

In [ ]:
fig,ax = plt.subplots()
ax.plot(t_trunc, rmsds_trunc_A_100, label = 'replicate 1', color = 'lightskyblue', linewidth=0.5)
ax.plot(t_trunc, rmsds_trunc_2_A_100, label = 'replicate 2', color = 'thistle', linewidth=0.5)
ax.plot(t_trunc, rmsds_trunc_3_A_100, label = 'replicate 3', color = 'yellowgreen', linewidth=0.5)
ax.plot(t_trunc, rmsd_100_avg, label = 'mean', color = 'black', linewidth=1.2)

ax.set(xlabel='t (ns)', ylabel='$RMSD_{100}$ ($\AA$)', title=system_code)
#ax.grid()
ax.legend(bbox_to_anchor=(1,1), loc='upper left')
ax.set_xlim([0.0, 200.0])
ax.set_ylim([0.0, 10.0])

plt.gcf().set_size_inches(10,5)

fig.savefig(work_dir+'analysis/rmsd-trunc-100-mean.png', bbox_inches="tight")

plt.show()

In [ ]:
f = open(work_dir+'rmsd_100_avg.txt', 'w')
for i in rmsd_100_avg:
    f.write(str(i)+'\n')
f.close()

In [ ]:
np.mean(rmsd_100_avg)

In [ ]:
rmsd_100_tuplelist = []

for i in range(len(rmsds_trunc_A_100)):
    rmsd_100_tuplelist.append((rmsds_trunc_A_100[i],rmsds_trunc_2_A_100[i],rmsds_trunc_3_A_100[i]))

In [ ]:
mean_list = []
std_list = []

for tupl in rmsd_100_tuplelist:
    mean_list.append(np.mean(tupl))
    std_list.append(np.std(tupl))

In [ ]:
fig,ax = plt.subplots()

ax.set(xlabel='t (ns)', ylabel='$RMSD_{100}$ ($\AA$)', title=system_code)
#ax.grid()

ax.errorbar(t_trunc, mean_list, yerr = std_list, fmt = 'none', ecolor='lightgrey',elinewidth = 1, capsize = 1)
ax.plot(t_trunc, mean_list, color = 'black', linewidth=1.2)

#ax.legend(bbox_to_anchor=(1,1), loc='upper left')
ax.set_xlim([0.0, 200.0])
ax.set_ylim([0.0, 10.0])

plt.gcf().set_size_inches(10,5)

fig.savefig(work_dir+'analysis/rmsd-trunc-100-with-error.png', bbox_inches="tight")

plt.show()

### RMSF calculation

In [ ]:
rmsfs = md.rmsf(protein_traj, protein_traj, 0, atom_indices = [a.index for a in protein_traj.topology.atoms if a.name == 'CA'])

In [ ]:
fig,ax = plt.subplots()
ax.plot(res_idx, rmsfs)
ax.set(xlabel='residue number', ylabel='RMSF (nm)', title='RMSF')
ax.grid()

fig.savefig(work_dir+'analysis/rmsf.png')
plt.show()

In [ ]:
fig,ax = plt.subplots()
ax.plot(res_idx[1:], rmsfs[1:])
ax.set(xlabel='residue number', ylabel='RMSF (nm)', title='RMSF')
ax.grid()

fig.savefig(work_dir+'analysis/rmsf-no-first-residue.png')
plt.show()

In [ ]:
fig,ax = plt.subplots()
ax.plot(chain_idx[0], rmsfs[0*chain_length:1*chain_length])
ax.set(xlabel='residue number', ylabel='RMSF (nm)', title='RMSF')
ax.grid()

fig.savefig(work_dir+'analysis/rmsf-per-chain-1.png')
plt.show()

In [ ]:
fig,ax = plt.subplots()
ax.plot(chain_idx[0][1:], rmsfs[0*chain_length:1*chain_length][1:])
ax.set(xlabel='residue number', ylabel='RMSF (nm)', title='RMSF')
ax.grid()

fig.savefig(work_dir+'analysis/rmsf-per-chain-1-no-first-residue.png')
plt.show()

RMSF for all chains

In [ ]:
fig,ax = plt.subplots()
ax.plot(chain_idx[0], rmsfs[0*chain_length:1*chain_length], label = "A")
ax.plot(chain_idx[0], rmsfs[1*chain_length:2*chain_length], label = "B")
ax.plot(chain_idx[0], rmsfs[2*chain_length:3*chain_length], label = "C")
ax.plot(chain_idx[0], rmsfs[3*chain_length:4*chain_length], label = "D")
#ax.plot(chain_idx[0], rmsfs[4*chain_length:5*chain_length], label = "E")
#ax.plot(chain_idx[0], rmsfs[5*chain_length:6*chain_length], label = "F")

ax.set(xlabel='residue number', ylabel='RMSF (nm)', title='RMSF')
ax.grid()
ax.legend(bbox_to_anchor=(1,1), loc='upper left')

fig.savefig(work_dir+'analysis/rmsf-per-chain-multiple.png', bbox_inches="tight")
plt.show()

In [ ]:
fig,ax = plt.subplots()

ax.plot(chain_idx[0][1:], rmsfs[0*chain_length:1*chain_length][1:], label = "A")
ax.plot(chain_idx[0], rmsfs[1*chain_length:2*chain_length], label = "B")
ax.plot(chain_idx[0], rmsfs[2*chain_length:3*chain_length], label = "C")
ax.plot(chain_idx[0], rmsfs[3*chain_length:4*chain_length], label = "D")
#ax.plot(chain_idx[0], rmsfs[4*chain_length:5*chain_length], label = "E")
#ax.plot(chain_idx[0], rmsfs[5*chain_length:6*chain_length], label = "F")


ax.set(xlabel='residue number', ylabel='RMSF (nm)', title=system_code)
#ax.grid()
ax.set_xlim([1.0,30.0])
ax.legend(bbox_to_anchor=(1,1), loc='upper left')

fig.savefig(work_dir+'analysis/rmsf-per-chain-multiple-no-first-residue.png', bbox_inches="tight")
plt.show()

Comparisons between replicates

In [ ]:
rmsfs_2 = md.rmsf(protein_traj_2, protein_traj_2, 0, atom_indices = [a.index for a in protein_traj_2.topology.atoms if a.name == 'CA'])

rmsfs_3 = md.rmsf(protein_traj_3, protein_traj_3, 0, atom_indices = [a.index for a in protein_traj_3.topology.atoms if a.name == 'CA'])

In [ ]:
fig,ax = plt.subplots()
ax.plot(res_idx[1:], rmsfs[1:], label='replicate 1')
ax.plot(res_idx[1:], rmsfs_2[1:], label='replicate 2', color='darkorange')
ax.plot(res_idx[1:], rmsfs_3[1:], label='replicate 3', color='yellowgreen')

ax.set(xlabel='residue number', ylabel='RMSF (nm)', title='RMSF')
ax.grid()
ax.legend(bbox_to_anchor=(1,1), loc='upper left')

fig.savefig(work_dir+'analysis/rmsf-comparison.png', bbox_inches='tight')
plt.show()

In [ ]:
rmsf_avg_A = []
rmsf_A = []
rmsf_avg_B = []
rmsf_B = []
rmsf_avg_C = []
rmsf_C = []
rmsf_avg_D = []
rmsf_D = []
rmsf_avg_E = []
rmsf_E =[]
rmsf_avg_F = []
rmsf_F = []
   
for i in range(len(rmsfs[0*chain_length:1*chain_length][1:])):
    rmsf_A.append([rmsfs[0*chain_length:1*chain_length][1:][i], rmsfs_2[0*chain_length:1*chain_length][1:][i], rmsfs_3[0*chain_length:1*chain_length][1:][i]])

for i in rmsf_A:
    rmsf_avg_A.append(10*np.mean(i)) #convert to Angstroms
    
for i in range(len(rmsfs[1*chain_length:2*chain_length])):
    rmsf_B.append([rmsfs[1*chain_length:2*chain_length][i], rmsfs_2[1*chain_length:2*chain_length][i], rmsfs_3[1*chain_length:2*chain_length][i]])

for i in rmsf_B:
    rmsf_avg_B.append(10*np.mean(i))
    
for i in range(len(rmsfs[2*chain_length:3*chain_length])):
    rmsf_C.append([rmsfs[2*chain_length:3*chain_length][i], rmsfs_2[2*chain_length:3*chain_length][i], rmsfs_3[2*chain_length:3*chain_length][i]])

for i in rmsf_C:
    rmsf_avg_C.append(10*np.mean(i))
    
for i in range(len(rmsfs[3*chain_length:4*chain_length])):
    rmsf_D.append([rmsfs[3*chain_length:4*chain_length][i], rmsfs_2[3*chain_length:4*chain_length][i], rmsfs_3[3*chain_length:4*chain_length][i]])

for i in rmsf_D:
    rmsf_avg_D.append(10*np.mean(i))
    
for i in range(len(rmsfs[4*chain_length:5*chain_length])):
    rmsf_E.append([rmsfs[4*chain_length:5*chain_length][i], rmsfs_2[4*chain_length:5*chain_length][i], rmsfs_3[4*chain_length:5*chain_length][i]])

for i in rmsf_E:
    rmsf_avg_E.append(10*np.mean(i))
    
for i in range(len(rmsfs[5*chain_length:6*chain_length])):
    rmsf_F.append([rmsfs[5*chain_length:6*chain_length][i], rmsfs_2[5*chain_length:6*chain_length][i], rmsfs_3[5*chain_length:6*chain_length][i]])

for i in rmsf_F:
    rmsf_avg_F.append(10*np.mean(i))

In [ ]:
fig,ax = plt.subplots()
ax.plot(chain_idx[0][1:], rmsf_avg_A, label='A')
ax.plot(chain_idx[0], rmsf_avg_B, label='B', color='darkorange')
ax.plot(chain_idx[0], rmsf_avg_C, label='C', color='yellowgreen')
ax.plot(chain_idx[0], rmsf_avg_D, label='D', color='darkorchid')
#ax.plot(chain_idx[0], rmsf_avg_E, label='E', color='crimson')
#ax.plot(chain_idx[0], rmsf_avg_F, label='F', color='peru')

ax.set(xlabel='residue number', ylabel='RMSF ($\AA$)', title=system_code)
#ax.grid()
#ax.set_ylim([0.0, 30.0])
ax.set_xlim([1, 30])
ax.legend(bbox_to_anchor=(1,1), loc='upper left')

fig.savefig(work_dir+'analysis/rmsf-all-chains-and-replicates.png', bbox_inches='tight')
plt.show()

### Secondary structure calculation

In [ ]:
sec_struct = md.compute_dssp(protein_traj, simplified = False)
print(sec_struct)

In [ ]:
sec_struct_simple = md.compute_dssp(protein_traj, simplified=True)
print(sec_struct_simple)

In [ ]:
frames = len(sec_struct_simple)
print('the number of frames is', frames)

In [ ]:
idx = 0
struct_list=[]
resid_list=[]
aa = len(all_residues) #the number of total residues in the assembly

for idx in range(0, aa):
    for frame in sec_struct_simple:
        resid_list.append(frame[idx])
    struct_list.append(resid_list)
    resid_list=[]

In [ ]:
count = 0
helicity_per_residue = []
helicity = 0

for res in struct_list:
    if 'NA' in res:
        print('excluding the ACE and NH2 caps')   
    else:
        for snapshot in res:
            if snapshot == 'H':
                count+=1
            helicity = (count/frames)*100
        helicity_per_residue.append(helicity)
        count=0

In [ ]:
fig,ax = plt.subplots()
ax.plot(res_idx, helicity_per_residue)
ax.set(xlabel='residue number', ylabel='%helicity per residue', title='%helicity per residue')
ax.grid()

fig.savefig(work_dir+'analysis/hpr.png')
plt.show()

In [ ]:
fig,ax = plt.subplots()
ax.plot(chain_idx[0], helicity_per_residue[0*chain_length:1*chain_length])
ax.set(xlabel='residue number', ylabel='%helicity per residue', title='%helicity per residue')
ax.grid()

fig.savefig(work_dir+'analysis/hpr-per-chain-1.png')
plt.show()

In [ ]:
fig,ax = plt.subplots()
ax.plot(chain_idx[0], helicity_per_residue[0*chain_length:1*chain_length], label = "A")
ax.plot(chain_idx[0], helicity_per_residue[1*chain_length:2*chain_length], label = "B")
ax.plot(chain_idx[0], helicity_per_residue[2*chain_length:3*chain_length], label = "C")
ax.plot(chain_idx[0], helicity_per_residue[3*chain_length:4*chain_length], label = "D")
#ax.plot(chain_idx[0], helicity_per_residue[4*chain_length:5*chain_length], label = "E")
#ax.plot(chain_idx[0], helicity_per_residue[5*chain_length:6*chain_length], label = "F")

ax.set(xlabel='residue number', ylabel='%helicity per residue', title='%helicity per residue')
ax.grid()
ax.legend(bbox_to_anchor=(1,1), loc='upper left')

fig.savefig(work_dir+'analysis/hpr-per-chain-multiple.png', bbox_inches="tight")
plt.show()

Comparison between different replicates

In [ ]:
sec_struct_simple_2 = md.compute_dssp(protein_traj_2, simplified=True)
print(sec_struct_simple_2)

idx = 0
struct_list=[]
resid_list=[]
aa = len(all_residues) #the number of total residues in the assembly

for idx in range(0, aa):
    for frame in sec_struct_simple_2:
        resid_list.append(frame[idx])
    struct_list.append(resid_list)
    resid_list=[]

count = 0
helicity_per_residue_2 = []
helicity = 0

for res in struct_list:
    if 'NA' in res:
        print('excluding the ACE and NH2 caps')   
    else:
        for snapshot in res:
            if snapshot == 'H':
                count+=1
            helicity = (count/frames)*100
        helicity_per_residue_2.append(helicity)
        count=0


In [ ]:
sec_struct_simple_3 = md.compute_dssp(protein_traj_3, simplified=True)
print(sec_struct_simple_3)

idx = 0
struct_list=[]
resid_list=[]
aa = len(all_residues) #the number of total residues in the assembly

for idx in range(0, aa):
    for frame in sec_struct_simple_3:
        resid_list.append(frame[idx])
    struct_list.append(resid_list)
    resid_list=[]

count = 0
helicity_per_residue_3 = []
helicity = 0

for res in struct_list:
    if 'NA' in res:
        print('excluding the ACE and NH2 caps')   
    else:
        for snapshot in res:
            if snapshot == 'H':
                count+=1
            helicity = (count/frames)*100
        helicity_per_residue_3.append(helicity)
        count=0

In [ ]:
hel_avg_A = []
hel_A = []
hel_avg_B = []
hel_B = []
hel_avg_C = []
hel_C = []
hel_avg_D = []
hel_D = []
hel_avg_E = []
hel_E = []
hel_avg_F = []
hel_F = []

for i in range(len(helicity_per_residue[0*chain_length:1*chain_length])):
    hel_A.append([helicity_per_residue[0*chain_length:1*chain_length][i], helicity_per_residue_2[0*chain_length:1*chain_length][i], helicity_per_residue_3[0*chain_length:1*chain_length][i]])

for i in hel_A:
    hel_avg_A.append(np.mean(i))

for i in range(len(helicity_per_residue[1*chain_length:2*chain_length])):
    hel_B.append([helicity_per_residue[1*chain_length:2*chain_length][i], helicity_per_residue_2[1*chain_length:2*chain_length][i], helicity_per_residue_3[1*chain_length:2*chain_length][i]])

for i in hel_B:
    hel_avg_B.append(np.mean(i))

for i in range(len(helicity_per_residue[2*chain_length:3*chain_length])):
    hel_C.append([helicity_per_residue[2*chain_length:3*chain_length][i], helicity_per_residue_2[2*chain_length:3*chain_length][i], helicity_per_residue_3[2*chain_length:3*chain_length][i]])

for i in hel_C:
    hel_avg_C.append(np.mean(i))
    
for i in range(len(helicity_per_residue[3*chain_length:4*chain_length])):
    hel_D.append([helicity_per_residue[3*chain_length:4*chain_length][i], helicity_per_residue_2[3*chain_length:4*chain_length][i], helicity_per_residue_3[3*chain_length:4*chain_length][i]])

for i in hel_D:
    hel_avg_D.append(np.mean(i))
    
for i in range(len(helicity_per_residue[4*chain_length:5*chain_length])):
    hel_E.append([helicity_per_residue[4*chain_length:5*chain_length][i], helicity_per_residue_2[4*chain_length:5*chain_length][i], helicity_per_residue_3[4*chain_length:5*chain_length][i]])

for i in hel_E:
    hel_avg_E.append(np.mean(i))

for i in range(len(helicity_per_residue[5*chain_length:6*chain_length])):
    hel_F.append([helicity_per_residue[5*chain_length:6*chain_length][i], helicity_per_residue_2[5*chain_length:6*chain_length][i], helicity_per_residue_3[5*chain_length:6*chain_length][i]])

for i in hel_F:
    hel_avg_F.append(np.mean(i))

In [ ]:
fig,ax = plt.subplots()
ax.plot(chain_idx[0], hel_avg_A, label='A')
ax.plot(chain_idx[0], hel_avg_B, label='B', color='darkorange')
ax.plot(chain_idx[0], hel_avg_C, label='C', color='yellowgreen')
ax.plot(chain_idx[0], hel_avg_D, label='D', color='darkorchid')
#ax.plot(chain_idx[0], hel_avg_E, label='E', color='crimson')
#ax.plot(chain_idx[0], hel_avg_F, label='F', color='peru')

ax.set(xlabel='residue number', ylabel='%helicity per residue', title=system_code)
#ax.grid()
ax.set_ylim([0.0, 105.0])
ax.set_xlim([1, 30])
ax.legend(bbox_to_anchor=(1,1), loc='upper left')

fig.savefig(work_dir+'analysis/hpr-all-chains-and-replicates.png', bbox_inches='tight')
plt.show()